# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [13]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

In [14]:
#driver = 'mysql+pymysql'
#ip = '34.65.10.136'
#username = 'data-students'
#password = 'iR0nH@cK-D4T4B4S3'
#db = 'retail_sales'
#connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

In [15]:
driver = 'mysql+pymysql'
ip = "127.0.0.1"
username = 'root'
password = 'root'
db = 'retail_sales'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

In [16]:
# Engine & Query
engine = create_engine(connection_string)
query = 'SELECT * FROM raw_sales'

# Database Request
raw_sales = pd.read_sql(query,engine)
raw_sales.head(10)

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0
5,2015-01-04,28,21842,149.0,1.0
6,2015-01-04,28,21881,299.0,1.0
7,2015-01-04,29,6930,2199.0,1.0
8,2015-01-04,29,10515,169.0,1.0
9,2015-01-04,29,8624,149.0,1.0


Data Cleaning

1) I created a copy of the dataset to keep the original unmodified

In [17]:
raw_sales2=raw_sales.copy()

2) I checked for missing values. There weren't any missing values in this dataset

In [18]:
raw_sales2.isnull().sum() # by using sum() I got the missing values per column

date            0
shop_id         0
item_id         0
item_price      0
item_cnt_day    0
dtype: int64

3) I dropped all columns that were irrelevant for my analysis

In [19]:

raw_sales2.drop("date", inplace=True, axis=1)


In [20]:
raw_sales2.head(10)

,shop_id,item_id,item_price,item_cnt_day
0,29,1469,1199.0,1.0
1,28,21364,479.0,1.0
2,28,21365,999.0,2.0
3,28,22104,249.0,2.0
4,28,22091,179.0,1.0
5,28,21842,149.0,1.0
6,28,21881,299.0,1.0
7,29,6930,2199.0,1.0
8,29,10515,169.0,1.0
9,29,8624,149.0,1.0


4) I grouped the data by shop_id in order to get the aggregates

In [22]:
raw_sales2=raw_sales2.groupby("shop_id").sum()

In [23]:
(raw_sales2.groupby("shop_id").sum()).head(10)

,item_id,item_price,item_cnt_day
shop_id,,,
2,966879,99070.5,81.0
3,335745,67443.0,33.0
4,498624,29361.0,39.0
5,620868,33138.0,45.0
6,1266894,116352.0,150.0
7,669045,52371.0,63.0
10,310137,22707.0,30.0
12,1647339,212196.4,216.0
14,421977,33456.0,51.0


5) I analized descriptive statistics in the dataset

In [26]:
raw_sales2.groupby("shop_id").max().describe()

,item_id,item_price,item_cnt_day
count,4.500000e+01,45.000000,45.000000
mean,1.125186e+06,104200.298222,111.466667
std,9.180210e+05,76781.421070,87.046905
min,2.000760e+05,9285.000000,18.000000
25%,6.024330e+05,47109.000000,60.000000
50%,7.985730e+05,85737.000000,78.000000
75%,1.266894e+06,135318.000000,150.000000
max,3.976677e+06,327864.000000,402.000000


With the aggregate per store table I calculated the revenue by store

In [11]:
raw_sales2["revenue"]=raw_sales2["item_price"]*raw_sales2["item_cnt_day"]

In [12]:
raw_sales2.head(10)

,shop_id,item_id,item_price,item_cnt_day,revenue
0,29,1469,1199.0,1.0,1199.0
1,28,21364,479.0,1.0,479.0
2,28,21365,999.0,2.0,1998.0
3,28,22104,249.0,2.0,498.0
4,28,22091,179.0,1.0,179.0
5,28,21842,149.0,1.0,149.0
6,28,21881,299.0,1.0,299.0
7,29,6930,2199.0,1.0,2199.0
8,29,10515,169.0,1.0,169.0
9,29,8624,149.0,1.0,149.0
